This is a draft of how duckdb should be used in this project

In [10]:

from pathlib import Path

data_dir = Path('../../data/')

learn pathlib

In [11]:
print(data_dir)
print(sorted(data_dir.glob('*.csv')))
test_file = data_dir / 'C_LC.csv'
print(test_file)

../../data
[PosixPath('../../data/AA_C.csv'), PosixPath('../../data/C_AA.csv'), PosixPath('../../data/C_LC.csv'), PosixPath('../../data/C_TA.csv'), PosixPath('../../data/LC_C.csv'), PosixPath('../../data/TA_C.csv')]
../../data/C_LC.csv


### creating duckdb database of csv file

In [12]:
# in config file
chromosome_row = 2
pos_row = 3
start_row = 4
centromeres = {1: (29,30), 2: (7,9), 3: (26,29), 4: (6,10), 5: (22,26)}

In [26]:
import duckdb
rel = duckdb.read_csv(test_file, all_varchar=True)
#rel.show()
#duckdb.sql("SELECT COLUMNS('^\\d+_+\\d') FROM rel")
query = r"""
WITH cleaned_rel AS (
    SELECT * FROM rel
    WHERE Sample IS NOT NULL AND Sample != ''
)
SELECT 
    Sample,
    split_part(header, '_', 1)::INTEGER AS chr,
    split_part(header, '_', 2)::INTEGER AS pos,
    val AS genotype
FROM cleaned_rel
UNPIVOT (
    val FOR header IN (COLUMNS('^\d+_+\d'))
)
"""

# Execute
tidy_data = duckdb.sql(query)
tidy_data.show()

┌─────────┬───────┬───────┬──────────┐
│ Sample  │  chr  │  pos  │ genotype │
│ varchar │ int32 │ int32 │ varchar  │
├─────────┼───────┼───────┼──────────┤
│ C-LC-1  │     1 │     1 │ A        │
│ C-LC-1  │     1 │     2 │ A        │
│ C-LC-1  │     1 │     3 │ A        │
│ C-LC-1  │     1 │     4 │ A        │
│ C-LC-1  │     1 │     5 │ A        │
│ C-LC-1  │     1 │     6 │ A        │
│ C-LC-1  │     1 │     7 │ A        │
│ C-LC-1  │     1 │     8 │ A        │
│ C-LC-1  │     1 │     9 │ A        │
│ C-LC-1  │     1 │    10 │ A        │
│   ·     │     · │     · │ ·        │
│   ·     │     · │     · │ ·        │
│   ·     │     · │     · │ ·        │
│ C-LC-28 │     5 │     7 │ H        │
│ C-LC-28 │     5 │     8 │ H        │
│ C-LC-28 │     5 │     9 │ H        │
│ C-LC-28 │     5 │    10 │ H        │
│ C-LC-28 │     5 │    11 │ H        │
│ C-LC-28 │     5 │    12 │ H        │
│ C-LC-28 │     5 │    13 │ H        │
│ C-LC-28 │     5 │    14 │ H        │
│ C-LC-28 │     5 │    15

In [23]:
duckdb.sql("COPY tidy_data TO 'test_output.csv'")